##  [MangoPlate Crawling](https://www.mangoplate.com/)    
- **'강남 칵테일' 페이지 10개 크롤링**   

In [1]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.common.keys import Keys

In [2]:
# 음식점 1개에서 정보 크롤링 함수
def FoodCrawling(row, col):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    
    for i in range(row):
        driver.execute_script("window.scrollTo(0, 400)")
        for j in range(col):
            # 동영상 1개 담을 그릇
            sub_dict = {}
            
            # 음식점 클릭
            element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > a > div > img'
            driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.CSS_SELECTOR, element))
            driver.find_element(By.CSS_SELECTOR, element).click()
            time.sleep(0.5)

            # 가게 이름 크롤링
            element = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
            title_raw = driver.find_element(By.CSS_SELECTOR, element)
            title = title_raw.text

            # 가게 전체 평점 점수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
                total_raw = driver.find_element(By.CSS_SELECTOR, element)
                total = total_raw.text
            except:
                total = '0'
            
            # 조회수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR, element)
                view = view_raw.text
            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
                view_raw = driver.find_element(By.CSS_SELECTOR, element)
                view = view_raw.text

            # 리뷰 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_review = review_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
                review_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_review = review_raw.text

            # 별표 개수 크롤링
            try:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_star = star_raw.text

            except:
                element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
                star_raw = driver.find_element(By.CSS_SELECTOR, element)
                num_star = star_raw.text
                
            # 주소 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(1) > td'
            address_raw = driver.find_element(By.CSS_SELECTOR, element)
            address = address_raw.text
            
            # 전화번호 크롤링
            element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child(2) > td'
            telephone_raw = driver.find_element(By.CSS_SELECTOR, element)
            telephone = telephone_raw.text
            
            # 이미지 크롤링
            element = 'body > main > article > aside.restaurant-photos > div > div.owl-wrapper-outer > div > div:nth-child(1) > figure > figure > img'
            image_raw = driver.find_element(By.CSS_SELECTOR, element)
            image = image_raw.get_attribute('src')
            
            # sub_dict에 담기
            sub_dict['title'] = title
            sub_dict['point'] = total
            sub_dict['hit'] = view
            sub_dict['review'] = num_review
            sub_dict['star'] = num_star
            sub_dict['address'] = address
            sub_dict['telephone'] = telephone
            sub_dict['image'] = image
            
             # total_dict에 담기
            total_dict[k] = sub_dict
            k += 1
            time.sleep(1)
            
            # 뒤로 가기
            driver.back()
            time.sleep(0.5)
            
    df = pd.DataFrame.from_dict(total_dict, orient='index')
    driver.close()
    return df

In [3]:
# 키워드를 입력하기
keyword = input('지역, 식당 또는 음식을 입력하세요:) ')

지역, 식당 또는 음식을 입력하세요:) 강남 칵테일


In [4]:
# 모든 데이터 프레임 담을 그릇
king_dict = {}

# 1페이지부터 순차적으로 들어가 크롤링하기
pages = 10
for page in range(pages):    
    # 크롬창 띄우기
    chrome_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(chrome_path)
    driver.maximize_window() # 창 최대화
    driver.get(" https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page+1))
    time.sleep(2)
    
    king_dict[page] = FoodCrawling(row=10, col=2)
    time.sleep(0.5)

In [5]:
king_dict

{0:        title point      hit review   star  \
 0       사브서울   4.7   22,986     25    789   
 1        제스트   4.6   14,145     28    564   
 2        PDR   4.4   14,714     12    771   
 3        피아마   4.4    6,206      5    276   
 4   화이트와인개러지   4.3   20,435     17    943   
 5       파인앤코   4.3   12,927     14    519   
 6       임바이브   4.3    9,135      9    251   
 7      리커테이션   4.3   13,432     14    729   
 8         유희   4.3    3,154      7    139   
 9     로비라운지바   4.2   34,319     37    495   
 10     복싱타이거   4.2   18,530     21    668   
 11         획   4.2   10,481      8    422   
 12         작   4.2   13,031      9    417   
 13     장생건강원   4.1   22,983     81    977   
 14       르챔버   4.1  188,590     97  5,697   
 15     심퍼티쿠시   4.1   16,930     20    601   
 16    커피 바 K   4.1   38,293     57    984   
 17      와인웍스   4.1   22,833     11    687   
 18   봉쥬르와인살롱   4.1    7,194     10    101   
 19      와인소셜   4.1    3,547      7    178   
 
                             

In [6]:
# csv 파일로 저장
df1 = king_dict[0]
df2 = king_dict[1]
df3 = king_dict[2]
df4 = king_dict[3]
df5 = king_dict[4]
df6 = king_dict[5]
df7 = king_dict[6]
df8 = king_dict[7]
df9 = king_dict[8]
df10 = king_dict[9]

df1.to_csv('Gangnam_Cocktail1.csv', encoding='utf-8-sig', index=False)
df2.to_csv('Gangnam_Cocktail2.csv', encoding='utf-8-sig', index=False)
df3.to_csv('Gangnam_Cocktail3.csv', encoding='utf-8-sig', index=False)
df4.to_csv('Gangnam_Cocktail4.csv', encoding='utf-8-sig', index=False)
df5.to_csv('Gangnam_Cocktail5.csv', encoding='utf-8-sig', index=False)
df6.to_csv('Gangnam_Cocktail6.csv', encoding='utf-8-sig', index=False)
df7.to_csv('Gangnam_Cocktail7.csv', encoding='utf-8-sig', index=False)
df8.to_csv('Gangnam_Cocktail8.csv', encoding='utf-8-sig', index=False)
df9.to_csv('Gangnam_Cocktail9.csv', encoding='utf-8-sig', index=False)
df10.to_csv('Gangnam_Cocktail10.csv', encoding='utf-8-sig', index=False)

In [7]:
# 저장한 csv 불러오기
df1 = pd.read_csv('data_project/Gangnam_Cocktail1.csv')
df2 = pd.read_csv('data_project/Gangnam_Cocktail2.csv')
df3 = pd.read_csv('data_project/Gangnam_Cocktail3.csv')
df4 = pd.read_csv('data_project/Gangnam_Cocktail4.csv')
df5 = pd.read_csv('data_project/Gangnam_Cocktail5.csv')
df6 = pd.read_csv('data_project/Gangnam_Cocktail6.csv')
df7 = pd.read_csv('data_project/Gangnam_Cocktail7.csv')
df8 = pd.read_csv('data_project/Gangnam_Cocktail8.csv')
df9 = pd.read_csv('data_project/Gangnam_Cocktail9.csv')
df10 = pd.read_csv('data_project/Gangnam_Cocktail10.csv')

In [8]:
df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
df_all = pd.concat(df_list, ignore_index=True)
df_all

,title,point,hit,review,star,address,telephone,image
0,사브서울,4.7,"22,986",25,789,서울특별시 강남구 논현로175길 6 B1\n지번 서울시 강남구 신사동 580-5 B1,02-512-4939,https://mp-seoul-image-production-s3.mangoplat...
1,제스트,4.6,"14,145",28,564,서울특별시 강남구 도산대로55길 26 하늘빌딩 1F\n지번 서울시 강남구 청담동 8...,010-3177-8801,https://mp-seoul-image-production-s3.mangoplat...
2,PDR,4.4,"14,714",12,771,서울특별시 강남구 언주로134길 24 2F\n지번 서울시 강남구 논현동 116-1 2F,02-3444-8880,https://mp-seoul-image-production-s3.mangoplat...
3,피아마,4.4,"6,206",5,276,서울특별시 강남구 도산대로70길 9 2F\n지번 서울시 강남구 청담동 13 2F,0507-1307-5434,https://mp-seoul-image-production-s3.mangoplat...
4,화이트와인개러지,4.3,"20,435",17,943,서울특별시 강남구 삼성로149길 7 B1\n지번 서울시 강남구 청담동 12-15 B1,02-540-7560,https://mp-seoul-image-production-s3.mangoplat...
...,...,...,...,...,...,...,...,...
195,블랙바 스테이크하우스,0.0,"1,883",4,15,서울특별시 강남구 테헤란로 610 B1\n지번 서울시 강남구 대치동 995-16 B1,02-6012-7429,https://mp-seoul-image-production-s3.mangoplat...
196,시그넷드링킹앤더슨,0.0,"5,182",3,88,서울특별시 강남구 언주로 118\n지번 서울시 강남구 도곡동 467-24,02-576-6891,https://mp-seoul-image-production-s3.mangoplat...
197,버틀러,0.0,"2,363",3,41,서울특별시 강남구 논현로151길 41 상운빌딩 102호\n지번 서울시 강남구 신사동...,02-541-9587,https://mp-seoul-image-production-s3.mangoplat...
198,피헨,0.0,370,2,9,서울특별시 강남구 도산대로30길 39\n지번 서울시 강남구 논현동 75,02-543-7187,https://mp-seoul-image-production-s3.mangoplat...


In [9]:
# csv 파일 저장
df_all.to_csv('Gangnam_Cocktail_test.csv', encoding='utf-8-sig', index=False)

In [10]:
# 파일 불러오기
df_all = pd.read_csv('data_project/Gangnam_Cocktail_test.csv')

In [11]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      200 non-null    object 
 1   point      200 non-null    float64
 2   hit        200 non-null    object 
 3   review     200 non-null    int64  
 4   star       200 non-null    object 
 5   address    200 non-null    object 
 6   telephone  200 non-null    object 
 7   image      200 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 12.6+ KB


In [12]:
data = df_all.copy()

In [13]:
# address 데이터 정제
data['address'] = [str(x) for x in data['address']]
data['address'] = data['address'].str.replace('\n', '<br><br> ')

In [14]:
# telephone 데이터 정제
data['telephone'] = [str(x) for x in data['telephone']]
data['telephone'] = data['telephone'].str.replace('칵테일 / 와인', '000-0000-0000')

In [15]:
# hit 모든 원소들을 문자형으로 반환
data['hit'] = [str(x) for x in data['hit']]
data['hit'] = data['hit'].str.replace(',', '')
print(data['hit'].isna().sum(), '\n')
print(data['hit'])

0 

0      22986
1      14145
2      14714
3       6206
4      20435
       ...  
195     1883
196     5182
197     2363
198      370
199     3329
Name: hit, Length: 200, dtype: object


In [16]:
data['hit'] = data['hit'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      200 non-null    object 
 1   point      200 non-null    float64
 2   hit        200 non-null    int32  
 3   review     200 non-null    int64  
 4   star       200 non-null    object 
 5   address    200 non-null    object 
 6   telephone  200 non-null    object 
 7   image      200 non-null    object 
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 11.8+ KB


In [17]:
# star 모든 원소들을 문자형으로 반환
data['star'] = [str(x) for x in data['star']]
data['star'] = data['star'].str.replace(',', '')
print(data['star'].isna().sum(), '\n')
print(data['star'])

0 

0      789
1      564
2      771
3      276
4      943
      ... 
195     15
196     88
197     41
198      9
199     28
Name: star, Length: 200, dtype: object


In [18]:
data['star'] = data['star'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      200 non-null    object 
 1   point      200 non-null    float64
 2   hit        200 non-null    int32  
 3   review     200 non-null    int64  
 4   star       200 non-null    int32  
 5   address    200 non-null    object 
 6   telephone  200 non-null    object 
 7   image      200 non-null    object 
dtypes: float64(1), int32(2), int64(1), object(4)
memory usage: 11.1+ KB


In [19]:
data.dtypes

title         object
point        float64
hit            int32
review         int64
star           int32
address       object
telephone     object
image         object
dtype: object

In [20]:
#정제 완료한 데이터를 csv로 저장
data.to_csv('Gangnam_Cocktail.csv', encoding='utf-8-sig', index=False)